In [ ]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


from selenium.webdriver.support.wait import WebDriverWait
from datetime import datetime


import pandas as pd
import numpy as np
import time
import os

In [ ]:
website = 'indeed'

now = datetime.now()

start_date = now.strftime('%m%d%Y')

# make data/playlist/ directory
os.makedirs(website +'/' + start_date , exist_ok=True)

chrome_path = r'C:/Users/user/scraper_sel/chromedriver.exe' # /chromedriver.exe
download_dir = r"C:/Users/user/projects/scrape_salary/data"


# read selenium service desc. this indicate the location of the chromedriver
service = Service(executable_path=chrome_path)  # "./chromedriver.exe")

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_dir}

chrome_options.add_experimental_option('prefs', prefs)



driver = webdriver.Chrome(service=service, chrome_options=chrome_options)

url = r'http://ph.indeed.com/'

driver.get(url)
driver.implicitly_wait(8)


keyword = 'Data Science'
location = 'Philippines'

keyword_field = driver.find_element(By.ID, 'text-input-what')
keyword_field.clear()
keyword_field.send_keys(keyword)

location_field = driver.find_element(By.ID, 'text-input-where')
location_field.clear()
location_field.send_keys(location)

find = driver.find_element(By.XPATH, "//button[normalize-space()='Find jobs']")
find.click()


In [ ]:
page = 1

while True:


    print(f'Harvesting page: {page}....', end="")

    # mosaic-jobcards

    mosaic = driver.find_element(By.ID, 'mosaic-jobcards')\
                    .find_elements(By.CSS_SELECTOR, 'div[class*="job_seen_beacon"]')

    time.sleep(1)
    to_frame = []
    items = 0
    number = 0

    # create list container for clicking
    targets = driver.execute_script(
        "return document.getElementById('mosaic-jobcards').getElementsByClassName('jcs-JobTitle');"
        )

    for card in mosaic:

        job_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable(targets[number]))

        try:
            # JS click
            driver.execute_script('arguments[0].click();', job_button)
            items += 1

            try:
                summary = driver.find_element(By.ID, 'jobDescriptionText').text.strip()

            except:
                summary = None
            
        except:
            print(f'click {number} failed')
                
        location = card.find_element(By.CLASS_NAME, 'companyLocation').text.strip()
        title = card.find_element(By.CLASS_NAME, 'jobTitle').text.strip()
        company = card.find_element(By.CLASS_NAME, "companyName").text.strip()
        
        try:
            salary = card.find_element(By.CLASS_NAME, "salary-snippet-container").text.strip()
        except:
            salary = None

        # try to get url,
        job_link = card.find_element(By.CSS_SELECTOR, 'a[class="jcs-JobTitle"]')
        job_url = job_link.get_attribute('href')


        job_info = {
            'location': location,
            'title': title,
            'company': company,
            'salary': salary,
            'summary': summary,
            'job_url': job_url}

        to_frame.append(job_info)
        number += 1

    framed_df = pd.DataFrame(to_frame)
    framed_df.to_csv(website + '/' + start_date + '/page' + str(page) + '.csv', index=False)
    print('Saved!')

    page += 1

    try:
        next_page = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH,
                    "(//a[@aria-label='Next Page'])[1]"
                    )))

        driver.execute_script("arguments[0].click();", next_page)

    except:
        print(f"Exception hit, scaping ended at page {page}")
        break
               
